In [8]:
from textgenrnn import textgenrnn
import pandas
import numpy as np
import rhymes
import importlib
importlib.reload(rhymes)
import re

word_level_model = textgenrnn('full_dataset_word_level.hdf5',
                                  vocab_path='word_level_vocab.json',
                                  config_path='word_level_config.json')



In [9]:
#uses model to generate a list of lyrics
def gen_lyrics():

    word_level_model = textgenrnn('full_dataset_word_level.hdf5',
                                  vocab_path='word_level_vocab.json',
                                  config_path='word_level_config.json')

    lyrics = word_level_model.generate(1, temperature=1.0, max_gen_length=1000, return_as_list=True)
    
    return lyrics

#Input: A list of generated lines
#Output: dictionary where the keys are the rhymes and the values are a list of lines that rhyme
def post_process(lines):

    rhyme_dict = dict()

    #Generate of list of last words from each line
    for index, line in enumerate(lines):
        if (len(line) == 0):
            continue
        try:
            wordsInLine = line.split()
            curr_word = ""
            wordsInLine[-1] = wordsInLine[-1].lower()
            #remove puncuation
            for c in wordsInLine[-1]:
                if ord(c) >= 97 and ord(c) <= 122:
                    curr_word += c

            #only add words (remove any empty strings)
            if curr_word != "":
                IT_RHYMES_BABYYY = False
                for key in rhyme_dict.keys():
                    if (rhymes.doTheyRhyme(curr_word, key)):
                        rhyme_dict[key].append(index)
                        IT_RHYMES_BABYYY = True
                if (not IT_RHYMES_BABYYY):
                    rhyme_dict[curr_word] = [index]
        except:
            print (curr_word, ' was not found in the dictionary')
            continue

    return rhyme_dict

#Input dictionary of rhymes with their corresponding line number
#Output: sorts each list of lines in the dictionary
def sort_dict(rhymes):
    
    #loop through each key
    for key in rhymes.keys():
        #get the list of lines associated with the rhyme
        index_list = rhymes[key]
        length = len(index_list)
        if (length < 2):
            continue
        
        #get a list of each distance between the list of lines
        distances = []
        distances.append(index_list[1] - index_list[0])
        for i in range(1, length - 1):
            distances.append(min(index_list[i] - index_list[i - 1], index_list[i + 1] - index_list[i]))

        distances.append(index_list[length - 1] - index_list[length - 2])

        
        #do some cool sorting stuff where you sort distances and the list of lines in parallel
        distances, index_list = zip(*sorted(zip(distances, index_list)))
        distances, index_list = (list(t) for t in zip(*sorted(zip(distances, index_list))))
        print(index_list)
        print(distances)
        
        #add the sorted list back to the dictionary
        rhymes[key] = index_list



#Uses dictionary of rhyming lines to generate lyrics with a user's rhyming sequence
def out1(lines, rhyme_dictionary):
    print ('What rhyming scheme would you like to use? (A to Z)')
    rhyme_scheme = input()
    if (not re.fullmatch('[A-Z]+', rhyme_scheme)):
        print ('That is not a valid rhyming scheme.')
        exit()

    # count the required number of lines for each letter/section of the rhyming scheme
    rhyme_counts = dict()
    for char in rhyme_scheme:
        if char in rhyme_counts:
            rhyme_counts[char] += 1
        else:
            rhyme_counts[char] = 1
    print (rhyme_counts)

    # map each letter/section of the rhyming scheme to a rhyme in rhyme_dict
    used_rhymes = set()
    letter_to_rhyme = dict()
    for letter in rhyme_counts.keys():
        for rhyme in rhyme_dictionary.keys():
            # if the current rhyme has enough rhymes for the current letter in the rhyming scheme
            # AND the rhyme hasn't been used, use it
            if (len(rhyme_dictionary[rhyme]) >= rhyme_counts[letter] and rhyme not in used_rhymes):
                letter_to_rhyme[letter] = rhyme_dictionary[rhyme]
                used_rhymes.add(rhyme)
                break

    print (letter_to_rhyme)

    # print the results!
    rhyme_indices = dict()
    for char in rhyme_scheme:
        if (char not in rhyme_indices):
            print (lines[letter_to_rhyme[char][0]])
            rhyme_indices[char] = 1
        else:
            print (lines[letter_to_rhyme[char][rhyme_indices[char]]])
            rhyme_indices[char] += 1

In [10]:
lines = gen_lyrics()[0].splitlines()

In [11]:
rhymes = post_process(lines)
sort_dict(rhymes)

dont  was not found in the dictionary


In [31]:
out1(lines, rhymes)

What rhyming scheme would you like to use? (A to Z)
BBBBBAAA
{'B': 5, 'A': 3}
{'B': [26, 28, 32, 9, 86], 'A': [3, 37, 92]}
i'm myself to make my life us rather live without yo
you wanna take not on the crowd who says you could never know
i wanna know where it should be,, aye, aye, so o...
wake me with the earth below
go (go - wo - wo - wo - wo oh)
and faith through your eyes, just as long as shine
date lots of time.
my heart is painted on the love of the line


[74, 91]
[-17, -17]
[3, 37, 92]
[34, 34, 55]
[26, 28, 32, 9, 86]
[2, 2, 4, 17, 54]
[10, 59]
[49, 49]
[53, 57, 68, 17]
[4, 4, 11, 36]
[35, 50]
[15, 15]
[25, 73]
[48, 48]
[27, 39, 51]
[12, 12, 12]
[30, 45]
[15, 15]
[61, 70]
[9, 9]
[41, 64]
[23, 23]
[43, 54]
[11, 11]
[55, 72]
[17, 17]
[62, 67]
[5, 5]


In [17]:
list1 = [3,2,4,1, 1]
list2 = ['three', 'two', 'four', 'one', 'one2']
list1, list2 = zip(*sorted(zip(list1, list2)))
print(list1)
print(list2)

(1, 1, 2, 3, 4)
('one', 'one2', 'two', 'three', 'four')
